## [--- Day 10: Factory ---](https://adventofcode.com/2025/day/10)

In [ ]:
import re
import pulp
from collections import defaultdict, deque
from dataclasses import dataclass


@dataclass
class Machine:
    diagram: int
    buttons: list[int]
    adjacency_list: defaultdict[int, list[int]]
    joltage: tuple[int, ...]


def shortest_path(goal: int, visited: dict[int, tuple[int, int]]) -> list[int]:
    current = goal
    path: list[int] = []
    while current != 0:
        parent, button = visited[current]
        path.append(button)
        current = parent

    return path[::-1]


def BFS(goal: int, buttons: list[int]) -> list[int]:
    visited: dict[int, tuple[int, int]] = {0: (0, 0)}
    queue: deque[int] = deque([0])

    while queue:
        current = queue.popleft()
        for button in buttons:
            next_state = current ^ button
            if next_state not in visited:
                visited[next_state] = (current, button)

                if next_state == goal:  # Solution found
                    return shortest_path(next_state, visited)

                queue.append(next_state)

    return [-1]  # No solution found


def ILP(
    num_buttons: int, list_buttons: dict[int, list[int]], target: tuple[int, ...]
) -> int:
    problem = pulp.LpProblem("Minimise_Button_Presses", pulp.LpMinimize)
    variables = [
        pulp.LpVariable(f"button_{i}", cat="Integer", lowBound=0)
        for i in range(num_buttons)
    ]
    problem += pulp.lpSum(variables)
    for counter_idx, button_indices in list_buttons.items():
        problem += (
            pulp.lpSum(variables[i] for i in button_indices) == target[counter_idx]
        )

    problem.solve()
    if problem.status == pulp.LpStatusOptimal:
        result = pulp.value(problem.objective)
        if isinstance(result, (int, float)):
            return int(round(result))

    return -1  # No solution found


def solve() -> None:
    machines: list[Machine] = []
    pattern: re.Pattern = re.compile(r"(?:[\[\(\{])([\.#\d,]+)(?:[\]\)\}])")

    # Parse each input line into a tuple of bitmasked diagram and buttons, and a list of joltages
    with open("..\\data\\10.txt") as file:
        for line in file:
            matches = pattern.findall(line.strip())

            diagram: int = sum((1 << i for i, c in enumerate(matches[0]) if c == "#"))
            buttons: list[int] = []
            buttons_adjacency_list = defaultdict(list)
            for i, button in enumerate(matches[1:-1]):
                integer_value = 0
                for idx in button.split(","):
                    value = int(idx)
                    integer_value |= 1 << value
                    buttons_adjacency_list[value].append(i)

                buttons.append(integer_value)

            joltage: tuple[int, ...] = tuple(map(int, matches[-1].split(",")))

            machines.append(Machine(diagram, buttons, buttons_adjacency_list, joltage))

    total_presses_a = 0
    total_presses_b = 0
    for machine in machines:
        path = BFS(machine.diagram, machine.buttons)
        presses = len(path)
        total_presses_a += presses
        part_b_presses = ILP(
            len(machine.buttons), machine.adjacency_list, machine.joltage
        )
        total_presses_b += part_b_presses

    print(f"Total button presses for all machines (part a): {total_presses_a}")
    print(f"Total button presses for all machines (part b): {total_presses_b}")


solve()

Total button presses for all machines (part a): 409
Total button presses for all machines (part b): 15489


In [ ]:
# TODO: Improve performance with SciPy or implement a custom solver


random 8193
